### Installation

In [2]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

### Unsloth

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True # Use 4bit quantization to reduce memory usage.


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-2-7b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.6: Fast Llama patching. Transformers: 4.55.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/3.87G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/948 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.8.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html).

<a name="Train"></a>
### Train the model
We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`

In [6]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass


In [7]:

from datasets import load_dataset
dataset = load_dataset("json", data_files='/content/drive/MyDrive/LLM-TKIG/entity_extraction_instruction.json', split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/423 [00:00<?, ? examples/s]

In [8]:
dataset

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 423
})

In [9]:
from trl import SFTConfig, SFTTrainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/423 [00:00<?, ? examples/s]

In [10]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
6.369 GB of memory reserved.


In [11]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 423 | Num Epochs = 2 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 39,976,960 of 6,778,392,576 (0.59% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.943100
2,2.007900
3,1.995700
4,1.990800
5,2.054700
6,2.262000
7,1.960400
8,1.489300
9,1.937100
10,1.839000


In [12]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

2238.2085 seconds used for training.
37.3 minutes used for training.
Peak reserved memory = 6.369 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 43.206 %.
Peak reserved memory for training % of max memory = 0.0 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!



In [13]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "An unknown attacker is using a complex and powerful new malware loader in relatively unsophisticated and low-reward attacks, indicating they may not realize the potential capabilities of the malware they are deploying. The malware, Trojan.Verblecon, is being used in attacks that appear to have installing cryptocurrency miners on infected machines as their end goal. There are some indications the attacker may also be interested in stealing access tokens for chat app Discord. However, the capabilities of this malware indicate that it could be highly dangerous if leveraged in ransomware or espionage campaigns. Verblecon was first spotted by analysts from Symantec, a division ofBroadcom Software, in January 2022. This blog will detail the capabilities of the malware. The malware is loaded as a server-side polymorphic JAR file. The fact that the file is polymorphic means that, due to encryption and obfuscation, the code of the malware payload looks different each time it is downloaded. Attackers generally pack malware in this way in an effort to evade detection by security software. The malware samples analyzed by Symantec were fully obfuscated, in the code flow, strings, and symbols. The samples themselves may be based onpublicly available code. Once started, the malware checks its command-line arguments. It requires at least one command-line argument to execute, which could be the infection or campaign ID initially e.g. \"CSIDL_SYSTEM_DRIVE\\program files\\java\\jre1.8.0_301\\bin\\javaw.exe\" -jar \"CSIDL_PROFILE\\appdata\\local\\temp\\rpvbh.jar\" masonkhonsari \"CSIDL_SYSTEM_DRIVE\\program files\\java\\jre1.8.0_301\\bin\\javaw.exe\" -jar \"CSIDL_PROFILE\\appdata\\local\\temp\\rpvbh.jar\" 923ec15ffa4474ca7bf200bfb90e782d Additionally, it also attempts to determine if its own process is being debugged by checking for the following Java command-line arguments: Next, it attempts to detect if it is being opened in a virtual or sandbox environment, which would indicate it is likely being opened on a security researcher’s machine. First, it checks for the following directories: It also obtains the machine MAC address and attempts to check for the following prefixes, which may indicate the file is being opened on a virtual machine: Following those checks, it executes the following command to obtain a list of running processes: It then appears to check these processes against a set list: It then also checks for the following files: Next, it appears to check the user name against the following: Then it executes the following command: It is unclear how the output is processed, however, there are some strings that could be related to this or other registry checks: If satisfied with these checks, it may copy itself as one of the following files: And then create one of the following files to use as a loadpoint: [INFECTION_ID] is computed as follows: Then it periodically attempts to connect to the following URLs: [DGA_NAME] is apparently generated using the following method: The traffic generated by the malware looks like this: The server response appears as the below. Some of the strings in this response indicate that the attacker may be leveraging legitimate Cloudflare infrastructure to host some of their C&C infrastructure. The server response body above is an encrypted blob that contains a URL signed with an RSA key. This blob can be decrypted and validated as follows: The malware then starts communicating with the decoded URL by sending details about the infected computer: The request body contains the following information about the infected machine in encrypted form: The server has been observed to respond as follows: Where the response body can be decrypted as follows: The last term above contains the following string: Some samples of the malware are seen communicating with the following servers: Communication between the malware and servers is over HTTP or HTTPS and this communication appears to culminate with victims being directed to connect to the following: The payload is downloaded from the URL observed earlier: The payload is obfuscated in a similar way to the other samples, and also contains similar techniques to detect the virtualization environment, as well as other functionality. The core functionality is to download and execute a binary blob from the following URL: The blob is decrypted along with *.bin artifacts from the same host. The downloaded blob is then cached on the local filesystem (in re-encrypted form) and injected into %Windows%\\SysWow64\\dllhost.exe for execution. The injection is performed using com.sun.jna and doesn't use usual APIs for injection. The final payload (hardwick.bin) contains the following embedded URL pointing to a configuration file for a cryptocurrency miner: This indicates that the purpose of this activity was to install cryptocurrency mining software on victim machines. The evidence found on victim networks appears to indicate that the goal of the attacker was to install cryptocurrency mining software on victim machines. This would appear to be a relatively low-reward goal for the attacker given the level of effort that would have been required to develop this sophisticated malware. There are also indications that the attacker may be stealing Discord tokens and using these to advertise Trojanized videogame applications. We suspect they were stealing Discord tokens because some of the obfuscated strings refer to pathnames that are apparently related to Discord clients, specifically: Discord is a group chatting app that is particularly popular among the gaming community. Advertising Trojanized videogame applications via Discord is likely a redistribution channel for Trojan.Verblecon. Most of the infections we saw where this malware was used were on non-enterprise machines; we rarely see ransomware deployed on non-enterprise machines. Previous reportshave connected related domains to a single occurrence of ransomware, but the infrastructure may be shared with an unrelated actor. The similarities between that incident and the activity we observed includes: However, we do not have enough evidence to draw a definitive link between both these sets of activity. The activity we have seen carried out using this sophisticated loader indicates that it is being wielded by an individual who may not realize the capabilities of the malware they are using. However, if it fell into the hands of a more sophisticated actor the potential is there for this loader to be used for more serious attacks, including potentially ransomware and espionage campaigns. For the latest protection updates, please visit theSymantec Protection Bulletin. If an IOC is malicious and the file available to us, Symantec Endpoint products will detect and block that file. 32a9415daa7f37a93dd0b347461844673c0f5baf0c15c01ee48b147dadf28299 3688c249774cc9a28d2b9b316921cec842bb087c57f4733cf5866226fbe2aeed 5a4f6332ad08b35c055bb5e6dfddc79d2f7905e63fac7595efbedd0b27f12eb8 007f5898c52c3aa1c3dca6d3a30f28f5f72d9789fbb440ae656d88959f68e53e f3f4af5f5eae1a28ad5a01b56d71302a265bce17d2c87ce731edf440612818a6 hxxp://verble[.]software/styles.jar hxxps://jonathanhardwick[.]me/hardwick.jar hxxps://jonathanhardwick[.]me/hardwick.bin hxxps://jonathanhardwick[.]me/config.txt hxxp://test.verble[.]rocks/dorflersaladreviews.jar hxxp://test.verble[.]rocks/dorflersaladreviews.bin The Threat Hunter Team is a group of security experts within Broadcom whose mission is to investigate targeted attacks, drive enhanced protection in Symantec and Carbon Black products, and offer analysis that helps customers respond to attacks. We encourage you to share your thoughts on your favorite social platform.", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nAn unknown attacker is using a complex and powerful new malware loader in relatively unsophisticated and low-reward attacks, indicating they may not realize the potential capabilities of the malware they are deploying. The malware, Trojan.Verblecon, is being used in attacks that appear to have installing cryptocurrency miners on infected machines as their end goal. There are some indications the attacker may also be interested in stealing access tokens for chat app Discord. However, the capabilities of this malware indicate that it could be highly dangerous if leveraged in ransomware or espionage campaigns. Verblecon was first spotted by analysts from Symantec, a division ofBroadcom Software, in January 2022. This blog will detail the capabilities of the malware. The malware is loaded as a server-side po

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [15]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
  alpaca_prompt.format(
        "An unknown attacker is using a complex and powerful new malware loader in relatively unsophisticated and low-reward attacks, indicating they may not realize the potential capabilities of the malware they are deploying. The malware, Trojan.Verblecon, is being used in attacks that appear to have installing cryptocurrency miners on infected machines as their end goal. There are some indications the attacker may also be interested in stealing access tokens for chat app Discord. However, the capabilities of this malware indicate that it could be highly dangerous if leveraged in ransomware or espionage campaigns. Verblecon was first spotted by analysts from Symantec, a division ofBroadcom Software, in January 2022. This blog will detail the capabilities of the malware. The malware is loaded as a server-side polymorphic JAR file. The fact that the file is polymorphic means that, due to encryption and obfuscation, the code of the malware payload looks different each time it is downloaded. Attackers generally pack malware in this way in an effort to evade detection by security software. The malware samples analyzed by Symantec were fully obfuscated, in the code flow, strings, and symbols. The samples themselves may be based onpublicly available code. Once started, the malware checks its command-line arguments. It requires at least one command-line argument to execute, which could be the infection or campaign ID initially e.g. \"CSIDL_SYSTEM_DRIVE\\program files\\java\\jre1.8.0_301\\bin\\javaw.exe\" -jar \"CSIDL_PROFILE\\appdata\\local\\temp\\rpvbh.jar\" masonkhonsari \"CSIDL_SYSTEM_DRIVE\\program files\\java\\jre1.8.0_301\\bin\\javaw.exe\" -jar \"CSIDL_PROFILE\\appdata\\local\\temp\\rpvbh.jar\" 923ec15ffa4474ca7bf200bfb90e782d Additionally, it also attempts to determine if its own process is being debugged by checking for the following Java command-line arguments: Next, it attempts to detect if it is being opened in a virtual or sandbox environment, which would indicate it is likely being opened on a security researcher’s machine. First, it checks for the following directories: It also obtains the machine MAC address and attempts to check for the following prefixes, which may indicate the file is being opened on a virtual machine: Following those checks, it executes the following command to obtain a list of running processes: It then appears to check these processes against a set list: It then also checks for the following files: Next, it appears to check the user name against the following: Then it executes the following command: It is unclear how the output is processed, however, there are some strings that could be related to this or other registry checks: If satisfied with these checks, it may copy itself as one of the following files: And then create one of the following files to use as a loadpoint: [INFECTION_ID] is computed as follows: Then it periodically attempts to connect to the following URLs: [DGA_NAME] is apparently generated using the following method: The traffic generated by the malware looks like this: The server response appears as the below. Some of the strings in this response indicate that the attacker may be leveraging legitimate Cloudflare infrastructure to host some of their C&C infrastructure. The server response body above is an encrypted blob that contains a URL signed with an RSA key. This blob can be decrypted and validated as follows: The malware then starts communicating with the decoded URL by sending details about the infected computer: The request body contains the following information about the infected machine in encrypted form: The server has been observed to respond as follows: Where the response body can be decrypted as follows: The last term above contains the following string: Some samples of the malware are seen communicating with the following servers: Communication between the malware and servers is over HTTP or HTTPS and this communication appears to culminate with victims being directed to connect to the following: The payload is downloaded from the URL observed earlier: The payload is obfuscated in a similar way to the other samples, and also contains similar techniques to detect the virtualization environment, as well as other functionality. The core functionality is to download and execute a binary blob from the following URL: The blob is decrypted along with *.bin artifacts from the same host. The downloaded blob is then cached on the local filesystem (in re-encrypted form) and injected into %Windows%\\SysWow64\\dllhost.exe for execution. The injection is performed using com.sun.jna and doesn't use usual APIs for injection. The final payload (hardwick.bin) contains the following embedded URL pointing to a configuration file for a cryptocurrency miner: This indicates that the purpose of this activity was to install cryptocurrency mining software on victim machines. The evidence found on victim networks appears to indicate that the goal of the attacker was to install cryptocurrency mining software on victim machines. This would appear to be a relatively low-reward goal for the attacker given the level of effort that would have been required to develop this sophisticated malware. There are also indications that the attacker may be stealing Discord tokens and using these to advertise Trojanized videogame applications. We suspect they were stealing Discord tokens because some of the obfuscated strings refer to pathnames that are apparently related to Discord clients, specifically: Discord is a group chatting app that is particularly popular among the gaming community. Advertising Trojanized videogame applications via Discord is likely a redistribution channel for Trojan.Verblecon. Most of the infections we saw where this malware was used were on non-enterprise machines; we rarely see ransomware deployed on non-enterprise machines. Previous reportshave connected related domains to a single occurrence of ransomware, but the infrastructure may be shared with an unrelated actor. The similarities between that incident and the activity we observed includes: However, we do not have enough evidence to draw a definitive link between both these sets of activity. The activity we have seen carried out using this sophisticated loader indicates that it is being wielded by an individual who may not realize the capabilities of the malware they are using. However, if it fell into the hands of a more sophisticated actor the potential is there for this loader to be used for more serious attacks, including potentially ransomware and espionage campaigns. For the latest protection updates, please visit theSymantec Protection Bulletin. If an IOC is malicious and the file available to us, Symantec Endpoint products will detect and block that file. 32a9415daa7f37a93dd0b347461844673c0f5baf0c15c01ee48b147dadf28299 3688c249774cc9a28d2b9b316921cec842bb087c57f4733cf5866226fbe2aeed 5a4f6332ad08b35c055bb5e6dfddc79d2f7905e63fac7595efbedd0b27f12eb8 007f5898c52c3aa1c3dca6d3a30f28f5f72d9789fbb440ae656d88959f68e53e f3f4af5f5eae1a28ad5a01b56d71302a265bce17d2c87ce731edf440612818a6 hxxp://verble[.]software/styles.jar hxxps://jonathanhardwick[.]me/hardwick.jar hxxps://jonathanhardwick[.]me/hardwick.bin hxxps://jonathanhardwick[.]me/config.txt hxxp://test.verble[.]rocks/dorflersaladreviews.jar hxxp://test.verble[.]rocks/dorflersaladreviews.bin The Threat Hunter Team is a group of security experts within Broadcom whose mission is to investigate targeted attacks, drive enhanced protection in Symantec and Carbon Black products, and offer analysis that helps customers respond to attacks. We encourage you to share your thoughts on your favorite social platform.", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
An unknown attacker is using a complex and powerful new malware loader in relatively unsophisticated and low-reward attacks, indicating they may not realize the potential capabilities of the malware they are deploying. The malware, Trojan.Verblecon, is being used in attacks that appear to have installing cryptocurrency miners on infected machines as their end goal. There are some indications the attacker may also be interested in stealing access tokens for chat app Discord. However, the capabilities of this malware indicate that it could be highly dangerous if leveraged in ransomware or espionage campaigns. Verblecon was first spotted by analysts from Symantec, a division ofBroadcom Software, in January 2022. This blog will detail the capabilities of the malware. The malware is loaded as a server-side polymor

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [17]:
model.save_pretrained("/content/drive/MyDrive/LLM-TKIG/finetuned_model_entity_extractor")  # Local saving
tokenizer.save_pretrained("/content/drive/MyDrive/LLM-TKIG/finetuned_model_entity_extractor")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('/content/drive/MyDrive/LLM-TKIG/finetuned_model_entity_extractor/tokenizer_config.json',
 '/content/drive/MyDrive/LLM-TKIG/finetuned_model_entity_extractor/special_tokens_map.json',
 '/content/drive/MyDrive/LLM-TKIG/finetuned_model_entity_extractor/tokenizer.model',
 '/content/drive/MyDrive/LLM-TKIG/finetuned_model_entity_extractor/added_tokens.json',
 '/content/drive/MyDrive/LLM-TKIG/finetuned_model_entity_extractor/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
  from unsloth import FastLanguageModel
  model, tokenizer = FastLanguageModel.from_pretrained(
      model_name = "unsloth/llama-2-7b-bnb-4bit",
      max_seq_length = max_seq_length,
      dtype = dtype,
      load_in_4bit = load_in_4bit,
  )
  FastLanguageModel.for_inference(model) # Enable native 2x faster inference


inputs = tokenizer(
[
    alpaca_prompt.format(
        "An unknown attacker is using a complex and powerful new malware loader in relatively unsophisticated and low-reward attacks, indicating they may not realize the potential capabilities of the malware they are deploying. The malware, Trojan.Verblecon, is being used in attacks that appear to have installing cryptocurrency miners on infected machines as their end goal. There are some indications the attacker may also be interested in stealing access tokens for chat app Discord. However, the capabilities of this malware indicate that it could be highly dangerous if leveraged in ransomware or espionage campaigns. Verblecon was first spotted by analysts from Symantec, a division ofBroadcom Software, in January 2022. This blog will detail the capabilities of the malware. The malware is loaded as a server-side polymorphic JAR file. The fact that the file is polymorphic means that, due to encryption and obfuscation, the code of the malware payload looks different each time it is downloaded. Attackers generally pack malware in this way in an effort to evade detection by security software. The malware samples analyzed by Symantec were fully obfuscated, in the code flow, strings, and symbols. The samples themselves may be based onpublicly available code. Once started, the malware checks its command-line arguments. It requires at least one command-line argument to execute, which could be the infection or campaign ID initially e.g. \"CSIDL_SYSTEM_DRIVE\\program files\\java\\jre1.8.0_301\\bin\\javaw.exe\" -jar \"CSIDL_PROFILE\\appdata\\local\\temp\\rpvbh.jar\" masonkhonsari \"CSIDL_SYSTEM_DRIVE\\program files\\java\\jre1.8.0_301\\bin\\javaw.exe\" -jar \"CSIDL_PROFILE\\appdata\\local\\temp\\rpvbh.jar\" 923ec15ffa4474ca7bf200bfb90e782d Additionally, it also attempts to determine if its own process is being debugged by checking for the following Java command-line arguments: Next, it attempts to detect if it is being opened in a virtual or sandbox environment, which would indicate it is likely being opened on a security researcher’s machine. First, it checks for the following directories: It also obtains the machine MAC address and attempts to check for the following prefixes, which may indicate the file is being opened on a virtual machine: Following those checks, it executes the following command to obtain a list of running processes: It then appears to check these processes against a set list: It then also checks for the following files: Next, it appears to check the user name against the following: Then it executes the following command: It is unclear how the output is processed, however, there are some strings that could be related to this or other registry checks: If satisfied with these checks, it may copy itself as one of the following files: And then create one of the following files to use as a loadpoint: [INFECTION_ID] is computed as follows: Then it periodically attempts to connect to the following URLs: [DGA_NAME] is apparently generated using the following method: The traffic generated by the malware looks like this: The server response appears as the below. Some of the strings in this response indicate that the attacker may be leveraging legitimate Cloudflare infrastructure to host some of their C&C infrastructure. The server response body above is an encrypted blob that contains a URL signed with an RSA key. This blob can be decrypted and validated as follows: The malware then starts communicating with the decoded URL by sending details about the infected computer: The request body contains the following information about the infected machine in encrypted form: The server has been observed to respond as follows: Where the response body can be decrypted as follows: The last term above contains the following string: Some samples of the malware are seen communicating with the following servers: Communication between the malware and servers is over HTTP or HTTPS and this communication appears to culminate with victims being directed to connect to the following: The payload is downloaded from the URL observed earlier: The payload is obfuscated in a similar way to the other samples, and also contains similar techniques to detect the virtualization environment, as well as other functionality. The core functionality is to download and execute a binary blob from the following URL: The blob is decrypted along with *.bin artifacts from the same host. The downloaded blob is then cached on the local filesystem (in re-encrypted form) and injected into %Windows%\\SysWow64\\dllhost.exe for execution. The injection is performed using com.sun.jna and doesn't use usual APIs for injection. The final payload (hardwick.bin) contains the following embedded URL pointing to a configuration file for a cryptocurrency miner: This indicates that the purpose of this activity was to install cryptocurrency mining software on victim machines. The evidence found on victim networks appears to indicate that the goal of the attacker was to install cryptocurrency mining software on victim machines. This would appear to be a relatively low-reward goal for the attacker given the level of effort that would have been required to develop this sophisticated malware. There are also indications that the attacker may be stealing Discord tokens and using these to advertise Trojanized videogame applications. We suspect they were stealing Discord tokens because some of the obfuscated strings refer to pathnames that are apparently related to Discord clients, specifically: Discord is a group chatting app that is particularly popular among the gaming community. Advertising Trojanized videogame applications via Discord is likely a redistribution channel for Trojan.Verblecon. Most of the infections we saw where this malware was used were on non-enterprise machines; we rarely see ransomware deployed on non-enterprise machines. Previous reportshave connected related domains to a single occurrence of ransomware, but the infrastructure may be shared with an unrelated actor. The similarities between that incident and the activity we observed includes: However, we do not have enough evidence to draw a definitive link between both these sets of activity. The activity we have seen carried out using this sophisticated loader indicates that it is being wielded by an individual who may not realize the capabilities of the malware they are using. However, if it fell into the hands of a more sophisticated actor the potential is there for this loader to be used for more serious attacks, including potentially ransomware and espionage campaigns. For the latest protection updates, please visit theSymantec Protection Bulletin. If an IOC is malicious and the file available to us, Symantec Endpoint products will detect and block that file. 32a9415daa7f37a93dd0b347461844673c0f5baf0c15c01ee48b147dadf28299 3688c249774cc9a28d2b9b316921cec842bb087c57f4733cf5866226fbe2aeed 5a4f6332ad08b35c055bb5e6dfddc79d2f7905e63fac7595efbedd0b27f12eb8 007f5898c52c3aa1c3dca6d3a30f28f5f72d9789fbb440ae656d88959f68e53e f3f4af5f5eae1a28ad5a01b56d71302a265bce17d2c87ce731edf440612818a6 hxxp://verble[.]software/styles.jar hxxps://jonathanhardwick[.]me/hardwick.jar hxxps://jonathanhardwick[.]me/hardwick.bin hxxps://jonathanhardwick[.]me/config.txt hxxp://test.verble[.]rocks/dorflersaladreviews.jar hxxp://test.verble[.]rocks/dorflersaladreviews.bin The Threat Hunter Team is a group of security experts within Broadcom whose mission is to investigate targeted attacks, drive enhanced protection in Symantec and Carbon Black products, and offer analysis that helps customers respond to attacks. We encourage you to share your thoughts on your favorite social platform.", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
What is a famous tall tower in Paris?

### Input:


### Response:
One of the most famous and iconic tall towers in Paris is the Eiffel Tower. Standing at 324 meters (1,063 feet) tall, this wrought iron tower is a symbol of the city and a must-see attraction for tourists from all over the world.<|end_of_text|>
